In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ase #package called Atomic Simulation Environment which allows us to construct crystals easily
from ase.spacegroup import crystal
from ase.visualize import view 
from ase.visualize.plot import plot_atoms
from tqdm import tqdm
from ase.io import read
from ase.io import write

In [ ]:
'''

This script is designed to generate a large scale crystal meant to import into a LAMMPS script using the ASE package. 
The input involves searching up a .cif file corresponding to the unit cell structure of that particularly
desired crystal. This unit cell can be multiplied many times into a much larger supercell structure. 

@author: kmream
'''

In [116]:
# Read CIF file
atoms = read('1011176.cif')

# Replicate the unit cell
atoms *= (80, 70, 70)

view(atoms) 

write("quartz_supercell3.lmp", atoms, format="lammps-data", specorder=['Si', 'O'])

with open('quartz_supercell3.lmp', 'w') as f:
    f.write("ID Type x y z\n")
    for i, atom in enumerate(atoms, start=1):
        atom_type = atom.number  # or use a mapping from symbol if needed
        x, y, z = atom.position
        f.write(f"{i} {atom_type} {x:.6f} {y:.6f} {z:.6f}\n")  

In [117]:
write("quartz_supercell3.lmp", quartz, format="lammps-data", specorder=['Si', 'O'])

with open('quartz_supercell3.lmp', 'w') as f:
    f.write("ID Type x y z\n")
    for i, atom in enumerate(quartz, start=1):
        atom_type = atom.number  # or use a mapping from symbol if needed
        x, y, z = atom.position
        f.write(f"{i} {atom_type} {x:.6f} {y:.6f} {z:.6f}\n")  

In [113]:
data = pd.read_csv('quartz_supercell3.txt', sep = " ")
data['ID'] = data.index + 14
data.loc[data['Type'] == 14, 'Type'] = 1
data.loc[data['Type'] == 8, 'Type'] = 2
charge_map = {1: 1.0, 2: -0.5}
data['charge'] = data['Type'].map(charge_map)
charge_col = data.pop('charge')
data.insert(2, 'charge', charge_col)
data

,ID,Type,charge,x,y,z
0,14,1,1.0,2.307636,0.000000,0.000000
1,15,1,1.0,-1.153818,1.998471,1.801667
2,16,1,1.0,1.302682,2.256311,3.603333
3,17,1,1.0,-1.153818,1.998471,0.000000
4,18,1,1.0,2.307636,0.000000,3.603333
...,...,...,...,...,...,...
4115995,4116009,2,-0.5,172.740343,294.202489,375.388781
4115996,4116010,2,-0.5,172.251500,296.076295,377.190447
4115997,4116011,2,-0.5,169.795000,295.338516,377.707886
4115998,4116012,2,-0.5,168.416657,296.698770,375.906219


In [9]:
def write_lammps_data(df, filename):
    with open(filename, 'w') as f:
        f.write("#LAMMPS data file via pandas\n\n")
        f.write(f"{len(df)} atoms\n")
        f.write(f"{df['Type'].nunique() + 1} atom types\n\n")

        # Assuming orthogonal box and bounds can be inferred from coordinates
        xlo, xhi = df['x'].min(), df['x'].max()
        ylo, yhi = df['y'].min(), df['y'].max()
        zlo, zhi = df['z'].min(), df['z'].max()

        # Add some padding
        pad = 5.0
        f.write(f"{xlo - pad:.6f} {xhi + pad:.6f} xlo xhi\n")
        #f.write(f"{ylo - pad:.6f} {yhi + pad:.6f} ylo yhi\n")
        f.write(f"-300 300 ylo yhi\n")
        f.write(f"{zlo - pad:.6f} {zhi + pad:.6f} zlo zhi\n\n")
        
        f.write("Masses\n\n")
        f.write(f" 1 28.08500000 # Si\n")
        f.write(f" 2 15.99900000 # O\n")
        f.write(f" 3 196.9665500 # Au\n\n")

        f.write("Atoms\n\n")
        for _, row in df.iterrows():
            f.write(f"{int(row['ID'])} {int(row['Type'])} {int(row['charge'])} {row['x']} {row['y']} {row['z']}\n")

# Usage
#write_lammps_data(data, 'quartz_supercell2.lmp')

In [16]:
data = pd.read_csv('quartz_supercell3.txt', sep = " ")
data['ID'] = data.index + 14
data.loc[data['Type'] == 14, 'Type'] = 1
data.loc[data['Type'] == 8, 'Type'] = 2
charge_map = {1: 1.0, 2: -0.5}
data['charge'] = data['Type'].map(charge_map)
charge_col = data.pop('charge')
data.insert(2, 'charge', charge_col)
data

,ID,Type,charge,x,y,z
0,14,1,1.0,2.29810,0.000000,0.0000
1,15,1,1.0,-1.14905,1.990213,3.6000
2,16,1,1.0,1.30095,2.253311,1.8000
3,17,2,-0.5,1.35485,1.073612,0.6588
4,18,2,-0.5,0.25235,1.710140,2.9412
...,...,...,...,...,...,...
3527995,3528009,2,-0.5,218.30235,294.513329,375.5412
3527996,3528010,2,-0.5,221.34280,293.439718,376.8588
3527997,3528011,2,-0.5,218.89280,296.410185,373.7412
3527998,3528012,2,-0.5,220.75235,295.336573,375.0588


In [17]:
write_lammps_data(data, 'quartz_supercell3.lmp')